In [1]:
import keras
import numpy as np
import pretty_midi
import librosa
import glob
import os
import matplotlib.pyplot as plt
import librosa.display
import IPython.display as ipd
from train_util import NoteIsoSequence
from model import get_autoencoder, get_vae, get_gan
from train_util import program_map, one_hot_map
from keras import backend as K
import time
import sklearn
from hparams import *
import random

# synthesize without label and then add back for train?
# loud not at the end to force normalization?

Using TensorFlow backend.


In [2]:
midi_root_dir = "/home/faraaz/workspace/music-transcription/data/clean_midi/"
midi_files = glob.iglob(os.path.join(midi_root_dir, '**', '*.mid'))
midi_file = next(midi_files)
print(midi_file)

/home/faraaz/workspace/music-transcription/data/clean_midi/Redbone/Come and Get Your Love.mid


In [3]:
print("loading model")
model_type = 'ae+'

if model_type == 'ae':
    encoder = keras.models.load_model("ae/encoder-test.h")
    decoder = keras.models.load_model("ae/decoder-test.h")
    _, _, autoencoder = get_autoencoder(encoder, decoder)
if model_type == 'ae+':
    encoder = keras.models.load_model("ae+/encoder-5.h")
    classifier = keras.models.load_model("ae+/classifier-5.h")
#     decoder = keras.models.load_model("ae/decoder-test.h")
#     _, _, autoencoder = get_autoencoder(encoder, decoder)
elif model_type == 'vae':
    encoder = keras.models.load_model("vae/encoder-1.h")
    decoder = keras.models.load_model("vae/decoder-1.h")
    encoder, decoder, autoencoder, _ = get_vae(encoder, decoder)
#     autoencoder = keras.models.load_model('vae/vae-0.h', custom_objects={'my_vae_loss': my_vae_loss})
elif model_type == 'gan':
    autoencoder = keras.models.load_model('gan/ae-0.h')
else:
    assert False

loading model


/home/faraaz/anaconda3/envs/urop/lib/python3.6/site-packages/keras/engine/saving.py:292: UserWarning: No training configuration found in save file: the model was *not* compiled. Compile it manually.
  warnings.warn('No training configuration found in save file: '


In [4]:
test_batch_size = 4
test_generator = NoteIsoSequence(train_wav_files, sample_duration=sample_duration, 
                                 sample_rate=sample_rate, n_fft=n_fft, instr_indices=[],
                                 note_indices=[], song_indices=[], batch_size=test_batch_size)

In [5]:
test_i = np.random.randint(len(test_generator))
test_song_indices = test_generator.song_indices[test_i*test_batch_size:(test_i+1)*test_batch_size]
test_instr_indices = test_generator.instr_indices[test_i*test_batch_size:(test_i+1)*test_batch_size]
test_note_indices = test_generator.note_indices[test_i*test_batch_size:(test_i+1)*test_batch_size]

song_index = test_song_indices[0]
song = train_wav_files[song_index]
song = song[:-4] + ".mid"
print(song)
for i in range(test_batch_size):
    instr_index = test_instr_indices[i]
    note_index = test_note_indices[i]
    
    pm = pretty_midi.PrettyMIDI(song)
    instrument = pm.instruments[instr_index]
    note = instrument.notes[note_index]
    print("{}. {}".format(i, instrument))
    print("   {}".format(note))

/home/faraaz/workspace/music-transcription/data/clean_midi/Redbone/Come and Get Your Love.mid
0. Instrument(program=73, is_drum=False, name="MELODY")
   Note(start=181.762585, end=181.893536, pitch=62, velocity=96)
1. Instrument(program=0, is_drum=False, name="A.PIANO 1")
   Note(start=260.338788, end=261.392343, pitch=62, velocity=96)
2. Instrument(program=0, is_drum=False, name="A.PIANO 1")
   Note(start=259.761415, end=260.297122, pitch=76, velocity=96)
3. Instrument(program=73, is_drum=False, name="MELODY")
   Note(start=161.768838, end=162.268830, pitch=62, velocity=96)


In [6]:
print("predicting embeddings...")
embeddings = encoder.predict_generator(test_generator, steps=1)
print(len(embeddings))
print(embeddings.shape)
# predicted_decode = decoder.predict(predicted_encode)
# print(predicted_decode.shape)
# predicted_ae = autoencoder.predict_generator(test_generator, steps=1)
# print(predicted_ae.shape)
# assert np.all(predicted_decode == predicted_ae)
instr_preds = classifier.predict_generator(test_generator, steps=1)
print(len(instr_preds))
print(instr_preds.shape)

predicting embeddings...
4
(4, 64)
4
(4, 128)


In [7]:
n_clusters = 2
kmeans = sklearn.cluster.KMeans(n_clusters=n_clusters)
embed_clusters = kmeans.fit_predict(embeddings)
for i in range(test_batch_size):
    cluster_id = embed_clusters[i]
    instr_pred = np.argmax(instr_preds[i])
    instr_index = test_instr_indices[i]
    note_index = test_note_indices[i]
    
    pm = pretty_midi.PrettyMIDI(song)
    instrument = pm.instruments[instr_index]
    note = instrument.notes[note_index]
    print("{}. {}".format(i, instrument))
    print("   {}".format(note))
    print("   cluster_id {}".format(cluster_id))
    print("   instr_pred {}".format(instr_pred))
    

0. Instrument(program=73, is_drum=False, name="MELODY")
   Note(start=181.762585, end=181.893536, pitch=62, velocity=96)
   cluster_id 0
   instr_pred 0
1. Instrument(program=0, is_drum=False, name="A.PIANO 1")
   Note(start=260.338788, end=261.392343, pitch=62, velocity=96)
   cluster_id 1
   instr_pred 0
2. Instrument(program=0, is_drum=False, name="A.PIANO 1")
   Note(start=259.761415, end=260.297122, pitch=76, velocity=96)
   cluster_id 1
   instr_pred 0
3. Instrument(program=73, is_drum=False, name="MELODY")
   Note(start=161.768838, end=162.268830, pitch=62, velocity=96)
   cluster_id 0
   instr_pred 0


In [8]:
test_batch_size = 4
test_generator = NoteIsoSequence(train_wav_files, sample_duration=sample_duration, 
                                 sample_rate=sample_rate, n_fft=n_fft, instr_indices=[],
                                 note_indices=[], song_indices=[], batch_size=test_batch_size)
test_song_indices = test_generator.song_indices
test_instr_indices = test_generator.instr_indices
test_note_indices = test_generator.note_indices
test_instr_labels = np.array(test_generator.instr_labels)

In [9]:
print("predicting embeddings...")
start = time.time()
embeddings = encoder.predict_generator(test_generator)
print("{}s".format(time.time()-start))
print(len(embeddings))
print(embeddings.shape)
start = time.time()
instr_preds = classifier.predict_generator(test_generator)
print("{}s".format(time.time()-start))
print(len(instr_preds))
print(instr_preds.shape)

predicting embeddings...
63.08727526664734s
3136
(3136, 64)
63.29999089241028s
3136
(3136, 128)


In [10]:
n_clusters = 3
kmeans = sklearn.cluster.KMeans(n_clusters=n_clusters)
embed_clusters = kmeans.fit_predict(embeddings)
score = sklearn.metrics.adjusted_rand_score(test_instr_indices, embed_clusters)  
print(score)

0.957130605612955


In [11]:
n_clusters = 3
kmeans = sklearn.cluster.KMeans(n_clusters=n_clusters)
embed_clusters = kmeans.fit_predict(instr_preds)
score = sklearn.metrics.adjusted_rand_score(test_instr_indices, embed_clusters)  
print(score)

0.9723407527489492


/home/faraaz/anaconda3/envs/urop/lib/python3.6/site-packages/sklearn/cluster/k_means_.py:968: ConvergenceWarning: Number of distinct clusters (2) found smaller than n_clusters (3). Possibly due to duplicate points in X.
  return_n_iter=True)


In [16]:
label_preds = np.argmax(instr_preds, axis=1)
correct_preds = np.sum(np.count_nonzero(label_preds == test_instr_labels))
print("correct_preds {}".format(correct_preds))
print("total labels {}".format(len(test_instr_labels)))
print("%correct {}".format(correct_preds / len(test_instr_labels)))
print(np.count_nonzero(np.argmax(instr_preds, axis=1)))
print(np.count_nonzero(test_instr_labels))
print(np.argmax(instr_preds, axis=1))

correct_preds 3028
total labels 3136
%correct 0.9655612244897959
677
677
[ 0  0  0 ... 73  0  0]


In [17]:
for i in range(n_clusters):
    print("{}. {}".format(i, np.sum(np.count_nonzero(embed_clusters == i))))

0. 2459
1. 677
2. 0


In [19]:
for i in range(128):
    count = np.sum(np.count_nonzero(test_instr_labels == i))
    if count != 0:
        print("{}. {}".format(i, count))

0. 2459
26. 108
73. 569
